# Defining input parameters for a SKIRT simulation

Author: Alejandro Guzman-Ortega

The following code defines the most relevant input parameters for a SKIRT
simulation, formatting them in a template `.ski` file.

It is assumed that the input particle data (`stars.dat`and `dust.dat`) are
already available in the working directory, if not, take a look at 
`data_reduction.ipynb` notebook.

The template files were initially obtained with the SKIRT QA process and then
edited to have placeholders of the form `parameter='{parameter}'` for the
parameters that are to be set by the user. These are meant to be general-purpose
templates for simulations of FIRE2 galaxies, avoiding the need to create a
`.ski` file from scratch.

 The main components of the templates are:

- A `simulationMode` that is either `ExtinctionOnly` or `DustEmission`. For the
  latter case, secondary emission is not iterated over.
- A `Starburst99SEDFamily` for the stellar particles and a MW-like
  `WeingartnerDraineDustMix` for the medium.
- A `VoronoiMeshMedium` component for the dust distribution, along with a
  `VoronoiMeshSpatialGrid` for discretization of the dust distribution. For the
  latter case, the `policy` parameter is set to `ImportedSites` (which are taken
  from the positions set in the `dust.dat` file). Effectively this converts the gas particle data into a Voronoi tesselation grid as opposed to an octotree grid being fit to the particle data which has lower convergence.
- The instruments are SED + photometric images. For the `ExtinctionOnly` simulation mode the images are common UV to optical broad bands, while the `DustEmission` simulation mode includes IR broad bands. Check the list of `<BroadBand bandName="GALEX_GALEX_NUV"/>` in each template ski file for the exacy list of broad bands.

- Default paramaters for everything else. 


## What you need to know/set before running the main function

Before running the main function, you need to set some variables. These are:
- `snapnum`: The snapshot number of the FIRE2 object, used to infer the
  corresponding redshift.
- `simulation_object`: The simulation object name, used to set the appropriate
    cosmology parameters.
- `mass_fraction`: Set to 1 if you are directly importing dust mass from a simulation (i.e. the FIRE+dust sims from Caleb Choban). If you are not importing a dust mass this represents the fraction of metal mass to assume in dust (i.e. a dust-to-metals ratio MW~0.4). The exact equation SKIRT uses is dust mass = gas mass * metallicity * mass_fraction. 
- `pixel_scale`: The desired pixel scale for the SKIRT simulation, in arcseconds
  per pixel. I recommend to set this parameter to match the scale of the
  instrument you plan to compare with from the beginning.
- `ski_template_filepath`: The path to the SKIRT template file. 
- `skirt_sim_mode`: The SKIRT simulation mode. This is either `'dust_emission'`
  or `'extinction_only'`. 
- `output_path`: The path to the output file, typically located within the run directory.

- `numpackets`: The number of photon packets to be used in the simulation.
- `num_wavelengths`: The number of wavelengths points to be used in the
  instruments. This number is only set to SED instruments.
- `min_wavelength` and `max_wavelength` are the minimum and maximum wavelengths
  to be used in the simulation instruments, in microns and in the frame of the
  observer. Again, these are used to  set limits to SED instruments, but the
  minimum value also sets the minimum wavelength for the sources. Note that
  these values are set in the frame of the observer.
- `half_fov_kpc`: The half field of view of the simulation, in kpc. This is used to
  set the size of the simulation box, as well as the size of the image.


In [21]:
snapnum = 580
simulation_object = "m12i_res7100"
FIRE_ver=2
mass_fraction = 0.4 # fraction of metal mass to assume in dust
pixel_scale = 0.2  # arcsec / pixel
ski_template_filepath = "./templates/dust_emission_full_sb99_wdmix_mw_voronoi.ski"  # or extinction_only_full_sb99_wdmix_mw_voronoi.ski
skirt_sim_mode = "dust_emission"  # or extinction_only
output_path = "./rundir/run.ski"

numpackets = "1e7"
num_wavelengths = 200  # used for SEDs
min_wavelength = 0.1  # microns; observer frame
max_wavelength = 2000.0  # microns; observer frame
half_fov_kpc = 18.0

## Filling in the .`ski` template

Having set the parameters above, the main function will fill in the template.
Note that some other paramaters are hardcoded to 'sane' values within the
function. These are the arbitrary distance to the object (used for rest-frame
instruments) and the wavelength limits used for the wavelength grids that SKIRT
employs.

In [22]:
import numpy as np
import os
import astropy.units as u

from astropy.cosmology import LambdaCDM


def format_ski_file(
    snapnum: int,
    simulation_object: str,
    FIRE_ver: int,
    pixel_scale: float,
    ski_template_filepath: str,
    skirt_sim_mode: str,
    output_path: str,
    numpackets: str = "1e7",
    num_wavelengths: int = 200,
    min_wavelength: float = 0.09,
    max_wavelength: float = 2000.0,
    half_fov_kpc: float = 18.0,
    max_temperature: float = 0.0,
    mass_fraction: float = 1.0,
):
    """
    Prepare SKIRT input files for a given snapshot of a FIRE2 simulation.

    Parameters
    ----------
    snapnum : int
        Snapshot number to process.
    simulation_object : str
        Name of the simulation object (e.g., "m12i_res7100").
    FIRE_ver : int
        Version of FIRE simulation (e.g., 2,3) used for determine redshift of snapshot number.
    pixel_scale : float
        Pixel scale in arcseconds per pixel.
    ski_template_filepath : str
        Path to the SKIRT template file.
    skirt_sim_mode : str
        Simulation mode for SKIRT (e.g., "dust_emission" or "extinction_only").
    output_path : str
        Path to the output directory for SKIRT input files.
    numpackets : str, optional
        Number of packets for SKIRT simulation (default is "1e7").
    num_wavelengths : int, optional
        Number of wavelengths for SKIRT simulation (default is 200).
    min_wavelength : float, optional
        Minimum wavelength for SKIRT simulation (default is 0.09).
    max_wavelength : float, optional
        Maximum wavelength for SKIRT simulation (default is 2000.0).
    half_fov_kpc : float, optional
        Half field of view in kpc (default is 18.0).
    max_temperature : float
        Maximum temperature for the medium elements in the simulation. Unless
        using a custom dust prescription (e.g. with a constant dust-to-metals
        ratio), this should always be set to 0.0 K.
    mass_fraction : float
        Fraction of the total medium mass to include in the simulation. This
        should be set to 1.0 unless using a custom dust prescription (e.g. with a
        constant dust-to-metals ratio).
    """
    
    if FIRE_ver <=2: scale_factors = np.loadtxt("FIRE2_snapshot_scale-factors.txt")
    else: scale_factors = np.loadtxt("FIRE3_snapshot_scale-factors.txt")
    z = 1.0 / scale_factors[snapnum] - 1

    # For non-zero redshift snapshots, present-day observers are symbolized by an observer distance of zero. 
    # However, at z!=0 SKIRT with throw a fatal error. 
    # To get around this for the z=0 snapshot, override its redshift to a small value.
    # More info at https://skirt.ugent.be/root/_user_redshift.html
    # Important note for high-z, CMB heating of dust is off by default in this example.
    # To turn it on set includeHeatingByCMB="true" in DustEmissionOptions
    if snapnum == 600:
        z = 0.003 # corresponds to a luminosity distance of ~13 Mpc
        print(f"WARNING: Overriding redshift for snapnum 600 to {z = }.")

    # Define a cosmology object for a given FIRE2 object
    if simulation_object in ["m12i_res7100", "m10q_res7100"]:
        cosmo = LambdaCDM(
            name="AGORA",
            H0=70.2,
            Om0=0.272,
            Ode0=0.728,
            Ob0=0.0455,
            meta={
                "n_s": 0.961,
                "sigma8": 0.807,
                "reference": "Wetzel et al. 2023, Table 1",
            },
        )
    elif simulation_object in ["m11d_res7100", "m11i_res7100", "m11e_res7100"]:
        cosmo = LambdaCDM(
            name="PLANCK",
            H0=68.0,
            Om0=0.31,
            Ode0=0.69,
            Ob0=0.048,
            meta={
                "n_s": 0.97,
                "sigma8": 0.82,
                "reference": "Wetzel et al. 2023, Table 1",
            },
        )
    else:
        raise ValueError(f"Unknown simulation object: '{simulation_object}'. ")

    # Calculate the spatial scale in the simulation (in physical kpc) that
    # corresponds to 1 pixel
    kpc_per_pixel = (
        cosmo.kpc_proper_per_arcmin(z).to(u.kpc / u.arcsec).value * pixel_scale
    )

    # Set arbitrary distance to the object in Mpc
    d_to_source = 10.0  # only used for rest-frame instruments

    # Define limits for relevant wavelength ranges used in SKIRT

    # 1. Define *rest-frame* wavelength range for sources (stars)
    max_wavelength_source = 120.0  # microns; observer-frame
    min_wavelength_source, max_wavelength_source = (
        min_wavelength / (1 + z),
        max_wavelength_source / (1 + z),
    )

    # 2. Define wavelength limits for dust emission and radiation field
    #    grids; these are set to the same values reported in Camps & Baes (2020)
    num_wavelengths_dust_base = 100
    min_wavelength_dust_base, max_wavelength_dust_base = 0.2, 2e3

    num_wavelengths_dust_sub = 200
    min_wavelength_dust_sub, max_wavelength_dust_sub = 3.0, 25.0

    num_wavelengths_rad = 40
    min_wavelength_rad, max_wavelength_rad = 0.02, 10.0

    # 3. Define wavelength limits for *rest-frame* SED instruments
    min_wavelength_rest, max_wavelength_rest = (
        min_wavelength / (1 + z),
        max_wavelength / (1 + z),
    )

    # 4. Determine if metallicity should be imported
    # For simulations without live dust you want SKIRT to infer the dust mass 
    # from the gas mass and metallicity. Otherwise you can give SKIRT the exact
    # dust mass.
    # If mass_fraction < 1, dust mass is inferred from M_dust = M_gas * Z * mass_fraction
    # If mass_fraction = 1, dust mass is given directly by the simulation.
    import_metallicity = "true" if mass_fraction < 1.0 else "false"

    # -------

    # Read the SKIRT input file template
    with open(ski_template_filepath, "r") as f:
        ski_template = f.read()

    npixels = int(np.ceil(2 * half_fov_kpc / kpc_per_pixel))

    if npixels >= 1e4:
        raise ValueError(
            "Number of pixels must be less than 1e4. "
            + "The current redshift is too small or the pixel scale is too fine."
        )

    half_fov_pc = half_fov_kpc * 1e3
    fov_pc = 2 * half_fov_pc

    print(
        f"SKIRT simulation will use {npixels} x {npixels} pixels ({fov_pc:g} pc x {fov_pc:g} pc) "
        + f"with a physical scale of {kpc_per_pixel:.2f} kpc / pixel."
    )

    # Format the SKIRT input file with the appropriate parameters

    # This is a workaround to handle missing keys in the dictionary and avoid
    # KeyError exceptions. It returns the key wrapped in curly braces if the key
    # is missing.
    class SafeDict(dict):
        def __missing__(self, key):
            return "{" + key + "}"


    # Make output directory if it doesn't exist
    if not os.path.exists(os.path.dirname(output_path)):
        os.makedirs(os.path.dirname(output_path))

    with open(output_path, "w") as f:
        template = ski_template.format_map(
            SafeDict(
                numPackets=f"{numpackets:s}",
                redshift=f"{z:f}",
                reducedHubbleConstant=f"{cosmo.h:f}",
                matterDensityFraction=f"{cosmo.Om0:f}",
                minWavelengthSource=f"{min_wavelength_source:.4f}",
                maxWavelengthSource=f"{max_wavelength_source:.4f}",
                minWavelengthRad=f"{min_wavelength_rad:.4f}",
                maxWavelengthRad=f"{max_wavelength_rad:.4f}",
                numWavelengthsRad=f"{num_wavelengths_rad:d}",
                massFraction=f"{mass_fraction:.4f}",
                importMetallicity=f"{import_metallicity:s}",
                maxTemperature=f"{max_temperature:.4f}",
                minX=f"-{half_fov_pc:.4f}",
                maxX=f"{half_fov_pc:.4f}",
                minY=f"-{half_fov_pc:.4f}",
                maxY=f"{half_fov_pc:.4f}",
                minZ=f"-{half_fov_pc:.4f}",
                maxZ=f"{half_fov_pc:.4f}",
                distance=f"{d_to_source:.4f}",
                minWavelengthRest=f"{min_wavelength_rest:.4f}",
                maxWavelengthRest=f"{max_wavelength_rest:.4f}",
                numWavelengthsRest=f"{num_wavelengths:d}",
                minWavelengthObs=f"{min_wavelength:.4f}",
                maxWavelengthObs=f"{max_wavelength:.4f}",
                numWavelengthsObs=f"{num_wavelengths:d}",
                fieldOfViewX=f"{fov_pc:.4f}",
                numPixelsX=f"{npixels:d}",
                fieldOfViewY=f"{fov_pc:.4f}",
                numPixelsY=f"{npixels:d}",
                numPixelsZ=f"{npixels:d}",  # for probes
            )
        )

        if skirt_sim_mode == "dust_emission":
            template = template.format_map(
                SafeDict(
                    minWavelengthBaseGridDust=f"{min_wavelength_dust_base:.4f}",
                    maxWavelengthBaseGridDust=f"{max_wavelength_dust_base:.4f}",
                    numWavelengthsBaseGridDust=f"{num_wavelengths_dust_base:d}",
                    minWavelengthSubGridDust=f"{min_wavelength_dust_sub:.4f}",
                    maxWavelengthSubGridDust=f"{max_wavelength_dust_sub:.4f}",
                    numWavelengthsSubGridDust=f"{num_wavelengths_dust_sub:d}",
                )
            )

        f.write(template)

    print(f"Formatted SKIRT input file written to {output_path}")

In [23]:
format_ski_file(
    snapnum=snapnum,
    simulation_object=simulation_object,
    FIRE_ver=FIRE_ver,
    pixel_scale=pixel_scale,
    ski_template_filepath=ski_template_filepath,
    skirt_sim_mode=skirt_sim_mode,
    output_path=output_path,
    numpackets=numpackets,
    num_wavelengths=num_wavelengths,
    min_wavelength=min_wavelength,
    max_wavelength=max_wavelength,
    half_fov_kpc=half_fov_kpc,
    mass_fraction=mass_fraction,
)

0.4
SKIRT simulation will use 499 x 499 pixels (36000 pc x 36000 pc) with a physical scale of 0.07 kpc / pixel.
Formatted SKIRT input file written to ./rundir/run.ski


## Other relevant notes

### About wavelength ranges/grids

The maximum wavelength for the sources is set to **120.0 microns**, which is
fine since the overall wavelength range extends up to **2000.0 microns** for the
SED instruments. As a general rule, avoid launching photon packets with
wavelengths that won't be recorded by any instrument. Depending on the
instruments or filters being used, the wavelength range should be set
appropriately. Also, note that the initial limits are defined in the
**observer's frame**. For sources, these limits should be converted to the
**rest-frame** to properly sample the desired part of the spectrum. More on this
can be found in the [SKIRT documentation on wavelength
configuration](https://skirt.ugent.be/root/_user_wave_configure.html).

### The pixel scale and FOV

Although resampling to a desired pixel scale is possible after the simulation, I
recommend to set the pixel scale from the beginning, especially if you are
planning to compare with observations. Regarding the half-FOV, setting this
value to a multiple of the half-mass radius would be fine in most cases, as it
will capture the object well and avoid unnecessary large images. Values between
7.5 and 9.0 have worked well for me in the past. Note that higher resolution pixels 
will require higher photon numbers for proper convergence at the galaxy edge.
Check out the SKIRT [documentation](https://skirt.ugent.be/root/_user_statistics.html])
for reliability statistics. 
However this is not a strict requirement since it depends on your science case. For example,
the addition of background noise in post-processing can make these pixels below a real 
instruments signal-to-noise threshold.

### Instruments and probes

The templates include several instruments, both SED-only and Full (SED + image).
For some instruments, the distance is set to zero, which places them in the
observer frame at the redshift specified for the object (check [this section of
the docs](https://skirt.ugent.be/skirt9/class_distant_instrument.html)). In the
case of Full instruments, the wavelength range is defined using a set of
broadband filters. I've the full set of filters that
can be used with SKIRT (along with their corresponding pivot wavelengths) can be found [in this section of the docs](https://skirt.ugent.be/skirt9/class_broad_band.html). You
can also define custom filters, though they must follow a specific format (see
[this
section](https://skirt.ugent.be/skirt9/class_band.html#aec9dc7f74ddb47745115f4e031cff6b8)).
Regarding the probes, these are used to sample/visualize the physical components
of a given simulation (e.g. density, temperature). The templates also include a
set of these that I consider can be of interest. These are optional and can be
removed if not needed, as they are not part of the main simulation output. The
only one I recommend keeping is the `ConvergenceInfoProbe`, which reports the
convergence information of the simulation. 


### PAH Emission and High-z Observations

The default parameters in the template ski files are not set up for PAH emissions or high-z observations. This simulation assumes equilibrium thermal emission for grains `dustEmissionType="Equilibrium"` which only true for larger grains. PAHs are never in thermal equilibrium due to their small size so you will want to set this to nonequilibrium. For high-z observations, dust heating due to the CMB should be considered so you will want to set `includeHeatingByCMB="true"`.